<a href="https://colab.research.google.com/github/ZulfikarAkbar06/tugas1/blob/main/Praktikum_Big_Data_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praktikum3').getOrCreate()

data = [('Zul', 'Sales', 3000),
        ('Fikar', 'Sales', 4600),
        ('Nayla', 'Sales', 4100),
        ('Rahma', 'Finance', 3000)]
columns = ['Nama', 'Divisi', 'Gaji']

df = spark.createDataFrame(data, schema=columns)
df.show()

+-----+-------+----+
| Nama| Divisi|Gaji|
+-----+-------+----+
|  Zul|  Sales|3000|
|Fikar|  Sales|4600|
|Nayla|  Sales|4100|
|Rahma|Finance|3000|
+-----+-------+----+



In [ ]:
# membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praktikum3').getOrCreate()

data = [('Zul', 'Sales', 3000),
        ('Fikar', 'Sales', 4600),
        ('Nayla', 'Sales', 4100),
        ('Rahma', 'Finance', 3000)]
columns = ['Nama', 'Divisi', 'Gaji']

df = spark.createDataFrame(data, schema=columns)
df.show()

# Contoh operasi transformasi DataFrame
df.select('Nama', 'Gaji').show()
df.filter(df['Gaji'] > 3000).show()
df.groupBy('Divisi').avg('Gaji').show()
df.groupBy().sum("Gaji").show()

+-----+-------+----+
| Nama| Divisi|Gaji|
+-----+-------+----+
|  Zul|  Sales|3000|
|Fikar|  Sales|4600|
|Nayla|  Sales|4100|
|Rahma|Finance|3000|
+-----+-------+----+

+-----+----+
| Nama|Gaji|
+-----+----+
|  Zul|3000|
|Fikar|4600|
|Nayla|4100|
|Rahma|3000|
+-----+----+

+-----+------+----+
| Nama|Divisi|Gaji|
+-----+------+----+
|Fikar| Sales|4600|
|Nayla| Sales|4100|
+-----+------+----+

+-------+---------+
| Divisi|avg(Gaji)|
+-------+---------+
|  Sales|   3900.0|
|Finance|   3000.0|
+-------+---------+

+---------+
|sum(Gaji)|
+---------+
|    14700|
+---------+



In [ ]:
# membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praktikum3').getOrCreate()

data = [('Zul', 'Sales', 3000),
        ('Fikar', 'Sales', 4600),
        ('Nayla', 'Sales', 4100),
        ('Rahma', 'Finance', 3000)]
columns = ['Nama', 'Divisi', 'Gaji']

df = spark.createDataFrame(data, schema=columns)

# Contoh manipulasi tipe data kompleks
df = df.withColumn('Bonus Gaji', df['Gaji'] * 0.1)
df1 = df.withColumn('Total Gaji', df['Gaji'] + df['Bonus Gaji'])

df.show()
df1.show()

+-----+-------+----+----------+
| Nama| Divisi|Gaji|Bonus Gaji|
+-----+-------+----+----------+
|  Zul|  Sales|3000|     300.0|
|Fikar|  Sales|4600|     460.0|
|Nayla|  Sales|4100|     410.0|
|Rahma|Finance|3000|     300.0|
+-----+-------+----+----------+

+-----+-------+----+----------+----------+
| Nama| Divisi|Gaji|Bonus Gaji|Total Gaji|
+-----+-------+----+----------+----------+
|  Zul|  Sales|3000|     300.0|    3300.0|
|Fikar|  Sales|4600|     460.0|    5060.0|
|Nayla|  Sales|4100|     410.0|    4510.0|
|Rahma|Finance|3000|     300.0|    3300.0|
+-----+-------+----+----------+----------+



In [ ]:
# membuat DataFrame sederhana dan operasi dasar
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Praktikum3').getOrCreate()

data = [('Zul', 'Sales', 3000),
        ('Fikar', 'Sales', 4600),
        ('Nayla', 'Sales', 4100),
        ('Rahma', 'Finance', 3000)]
columns = ['Nama', 'Divisi', 'Gaji']

df = spark.createDataFrame(data, schema=columns)

# Contoh menggunakan window functions
from pyspark.sql.window import Window
from pyspark.sql import functions as F

windowSpec = Window.orderBy('Gaji')
df.withColumn('Tingkatan', F.rank().over(windowSpec)).show()

+-----+-------+----+---------+
| Nama| Divisi|Gaji|Tingkatan|
+-----+-------+----+---------+
|  Zul|  Sales|3000|        1|
|Rahma|Finance|3000|        1|
|Nayla|  Sales|4100|        3|
|Fikar|  Sales|4600|        4|
+-----+-------+----+---------+



In [8]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Buat Spark Session
spark = SparkSession.builder \
    .appName("MobileLegendsAnalysis") \
    .getOrCreate()

# Load dataset dari file CSV
df_reviews = spark.read.csv("mobile_legends_reviews.csv", header=True, inferSchema=True)

# Tampilkan skema dan data
df_reviews.printSchema()
df_reviews.show(5, truncate=False)

# Hitung distribusi rating
windowSpec = Window.orderBy("score") # Define a window spec
rating_distribution = df_reviews.groupBy("score").agg(
    F.count("*").alias("count"),
    F.round(F.count("*") / F.sum(F.count("*")).over(windowSpec) * 100, 2).alias("percentage")
).orderBy("score")

rating_distribution.show()

# Hitung rata-rata thumbs up per rating
avg_thumbsup_by_rating = df_reviews.groupBy("score").agg(
    F.avg("thumbsUpCount").alias("AvgThumbsUp")
).orderBy("score")

avg_thumbsup_by_rating.show()

# Analisis sentimen berdasarkan kata kunci umum
# Cari review yang menyebutkan "matchmaking" (case insensitive)
matchmaking_reviews = df_reviews.filter(F.lower(F.col("content")).contains("matchmaking"))
print(f"Jumlah review yang menyebutkan matchmaking: {matchmaking_reviews.count()}")

# Distribusi rating untuk review tentang matchmaking
matchmaking_rating_dist = matchmaking_reviews.groupBy("score").agg(
    F.count("*").alias("count")
).orderBy("score")

matchmaking_rating_dist.show()

# Cari review yang menyebutkan "dark system"
dark_system_reviews = df_reviews.filter(F.lower(F.col("content")).contains("dark system"))
print(f"Jumlah review yang menyebutkan dark system: {dark_system_reviews.count()}")

# Review dengan thumbs up terbanyak
most_liked_reviews = df_reviews.orderBy(F.desc("thumbsUpCount")).select("content", "score", "thumbsUpCount")
most_liked_reviews.show(10, truncate=50)

# Hitung persentase review positif vs negatif
positive_reviews = df_reviews.filter(F.col("score") >= 4).count()
negative_reviews = df_reviews.filter(F.col("score") <= 2).count()
neutral_reviews = df_reviews.filter(F.col("score") == 3).count()
total_reviews = df_reviews.count()

print(f"Total reviews: {total_reviews}")
print(f"Positive reviews (4-5 stars): {positive_reviews} ({positive_reviews/total_reviews*100:.2f}%)")
print(f"Neutral reviews (3 stars): {neutral_reviews} ({neutral_reviews/total_reviews*100:.2f}%)")
print(f"Negative reviews (1-2 stars): {negative_reviews} ({negative_reviews/total_reviews*100:.2f}%)")

# Analisis temporal - hitung jumlah review per hari
df_reviews = df_reviews.withColumn("date", F.to_date(F.substring(F.col("at"), 1, 10)))
reviews_per_day = df_reviews.groupBy("date").agg(F.count("*").alias("daily_reviews")).orderBy("date")
reviews_per_day.show()

# Tutup Spark Session
spark.stop()

root
 |-- reviewId: string (nullable = true)
 |-- userName: string (nullable = true)
 |-- content: string (nullable = true)
 |-- score: string (nullable = true)
 |-- thumbsUpCount: string (nullable = true)
 |-- at: string (nullable = true)

+------------------------------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+-------------+-------------------+
|reviewId                            |userName     |content                                                                                                                                                                                                                  